# load_ds: simple, medium, hard, each 5

In [ ]:
!pip install datasets

In [ ]:
import json
from datasets import Dataset, concatenate_datasets

In [ ]:
#with open("/content/LeetCodeDataset-v0.1.0-test.jsonl", encoding="utf-8") as f:
with open("LeetCodeDataset-v0.1.0-test.jsonl", encoding="utf-8") as f:
    rows = [json.loads(line) for line in f]

ds = Dataset.from_list(rows)
print(ds)

Dataset({
    features: ['task_id', 'prompt', 'completion', 'entry_point', 'test', 'query', 'response', 'input_output', 'meta'],
    num_rows: 175
})


In [ ]:
def first_k_by_difficulty(dataset, difficulty, k=5):
    """
    Return the first k rows whose meta['difficulty'] matches `difficulty`.
    """
    subset = dataset.filter(lambda ex: ex["meta"]["difficulty"] == difficulty)
    return subset.select(range(min(k, len(subset))))   # protects against < k rows



In [ ]:
easy_ds   = first_k_by_difficulty(ds, "Easy",   k=5)
medium_ds = first_k_by_difficulty(ds, "Medium", k=5)
hard_ds   = first_k_by_difficulty(ds, "Hard",   k=5)
example_ds = concatenate_datasets([easy_ds.select([0]), medium_ds.select([0]), hard_ds.select([0])]) # serve as few shot examples

Filter:   0%|          | 0/175 [00:00<?, ? examples/s]

Filter:   0%|          | 0/175 [00:00<?, ? examples/s]

Filter:   0%|          | 0/175 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


# load model

In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
if torch.backends.mps.is_built():
    device = "mps"
elif torch.backends.cuda.is_built():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
print(device)

cuda


In [ ]:
MODEL_NAME = "bigcode/starcoder2-3b"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device).eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

# docs + code gen pipe

In [ ]:
from itertools import islice
from textwrap import dedent

import re

## helpers

### build example_ds w/ DOCSTRINGS as part of the code

In [ ]:
# add docstring to code sol
def inject_docstrings(code: str, doc: str) -> str:

    lines = code.splitlines()

    # ── locate "class Solution" ─────────────────────────────────────────
    try:
        cls_idx = next(i for i, ln in enumerate(lines)
                       if re.match(r"\s*class\s+Solution\b", ln))
    except StopIteration:
        raise ValueError("No `class Solution` found in code.")

    cls_indent = re.match(r"\s*", lines[cls_idx]).group(0)

    # ── locate the first method header inside the class ────────────────
    # def_pattern = re.compile(rf"{cls_indent} {' ' * 4}(async\s+)?def\s+")
    def_pattern = re.compile(rf"{cls_indent}{' ' * 4}(async\s+)?def\s+")
    try:
        hdr_idx = next(i for i, ln in enumerate(lines[cls_idx + 1:], cls_idx + 1)
                       if def_pattern.match(ln))
    except StopIteration:
        raise ValueError("No method found under `class Solution`.")

    header_line = lines[hdr_idx]
    base_indent = re.match(r"\s*", header_line).group(0)
    body_indent = base_indent + " " * 4            # PEP-8

    # ── skip an existing docstring, if any ─────────────────────────────
    j = hdr_idx + 1
    if j < len(lines) and re.match(r'\s*[ru]?["\']{3}', lines[j]):
        quote = lines[j].lstrip()[:3]               # ''' or """
        j += 1
        while j < len(lines) and quote not in lines[j]:
            j += 1
        j += 1                                      # first code line after old docstring

    # ── build the new docstring block ──────────────────────────────────
    doc_block = [f'{body_indent}"""']
    doc_block += [f"{body_indent}{ln}" for ln in doc.strip().splitlines()]
    doc_block += [f'{body_indent}"""']

    # ── put the docs into that method ────────────────────────────────────────────
    new_lines = lines[: hdr_idx + 1] + doc_block + lines[j:]
    return "\n".join(new_lines)

In [ ]:
for ex in example_ds:
    print(ex['task_id'])

sum-of-squares-of-special-elements
find-the-longest-equal-subarray
maximum-sum-queries


In [ ]:
for ex in example_ds:
    print(ex['completion'])

class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
        n = len(nums)
        return sum(x * x for i, x in enumerate(nums, 1) if n % i == 0)

class Solution:
    def longestEqualSubarray(self, nums: List[int], k: int) -> int:
        cnt = Counter()
        l = 0
        mx = 0
        for r, x in enumerate(nums):
            cnt[x] += 1
            mx = max(mx, cnt[x])
            if r - l + 1 - mx > k:
                cnt[nums[l]] -= 1
                l += 1
        return mx

class BinaryIndexedTree:
    __slots__ = ["n", "c"]

    def __init__(self, n: int):
        self.n = n
        self.c = [-1] * (n + 1)

    def update(self, x: int, v: int):
        while x <= self.n:
            self.c[x] = max(self.c[x], v)
            x += x & -x

    def query(self, x: int) -> int:
        mx = -1
        while x:
            mx = max(mx, self.c[x])
            x -= x & -x
        return mx


class Solution:
    def maximumSumQueries(
        self, nums1: List[int], num

In [ ]:
DOCS_sumOfS = """
Compute the sum of squares of the *special* elements in a 1-indexed array.

        A “special” element is one whose 1-based index divides the length
        of the array.  Formally, for an array of length *n*, an element
        `nums[i]` (using 1-based indexing) is special if `n % i == 0`.

        Parameters
        ----------
        nums : List[int]
            A non-empty list of integers (1 ≤ len(nums) ≤ 50, 1 ≤ nums[i] ≤ 50).

        Returns
        -------
        int
            The sum of the squares of all special elements.

        Examples
        --------
        >>> Solution().sumOfSquares([1, 2, 3, 4])
        21
        # indices with 1-based indexing: 1, 2, 3, 4
        # n = 4 → divisors are 1, 2, 4 → elements = 1, 2, 4
        # 1² + 2² + 4² = 21

        >>> Solution().sumOfSquares([2, 7, 1, 19, 18, 3])
        63
        # n = 6 → divisors are 1, 2, 3, 6 → elements = 2, 7, 1, 3
        # 2² + 7² + 1² + 3² = 63
"""

DOCS_LESubarry = """
        Return the length of the longest contiguous sub-array that can be made
        **equal** (all elements identical) after deleting at most ``k`` elements.

        The algorithm slides a window across ``nums`` while maintaining:

        * ``cnt`` - a frequency counter of elements in the current window.
        * ``mx`` - the highest frequency within that window
          (i.e. the size of the majority element).
        * Window bounds ``[l, r]`` (both 0-based, inclusive).

        Because the minimum deletions required to make the window equal is
        ``window_size - mx``, we shrink the window from the left whenever that
        value exceeds ``k``.  The maximum ``mx`` observed over the scan is the
        answer.

        Parameters
        ----------
        nums : List[int]
            Input array (1 ≤ len(nums) ≤ 10⁵, 1 ≤ nums[i] ≤ len(nums)).
        k : int
            Maximum number of deletions allowed (0 ≤ k ≤ len(nums)).

        Returns
        -------
        int
            Length of the longest equal sub-array achievable with ≤ ``k`` deletions.

        Examples
        --------
        >>> Solution().longestEqualSubarray([1, 3, 2, 3, 1, 3], k=3)
        3
        >>> Solution().longestEqualSubarray([1, 1, 2, 2, 1, 1], k=2)
        4

        Complexity
        ----------
        Time : O(n)  - each index enters and leaves the window once.
        Space: O(u)  - ``u`` distinct values (≤ len(nums)) in the counter.
    """

DOCS_MAXSUM = """
        Answer each query ``[xi, yi]`` with the maximum value of
        ``nums1[j] + nums2[j]`` over all indices *j* such that
        ``nums1[j] ≥ xi`` **and** ``nums2[j] ≥ yi``; return -1 if
        no index satisfies the constraints.

        The algorithm processes queries **offline**:

        1.  Sort item pairs *(nums1, nums2)* by ``nums1`` descending.
        2.  Sort queries by ``xi`` descending; iterate through them.
        3.  As we lower the ``xi`` threshold, insert newly-eligible
            items into a Fenwick tree keyed by *compressed* ``nums2``
            values, storing the running maximum of ``nums1 + nums2``.
        4.  For a query's ``yi`` we binary-search the first index whose
            ``nums2`` ≥ ``yi`` and read the tree's prefix-max.

        Complexity
        ----------
        *Sorting*  ``O((n + q) · log n)``
        *Fenwick updates / queries*  ``≤ (n + q) · log n``
        *Space*  ``O(n)`` for the tree and sorted arrays.

        Parameters
        ----------
        nums1, nums2 : List[int]
            Parallel arrays of length *n* (``1 ≤ n ≤ 10⁵``).
        queries : List[List[int]]
            Each sub-list is ``[xi, yi]``.

        Returns
        -------
        List[int]
            Answers in the order the queries were given.
        """


In [ ]:
DOCS = [DOCS_sumOfS,DOCS_LESubarry, DOCS_MAXSUM]

### build few-shot part

In [ ]:
ONE_PROMPT_BLOCK_TEMPLATE = """<start>
{problem}
###
{examples}
###
<fim_prefix>
{prefix}
<fim_suffix>
<fim_middle>
{middle}
<end>""".strip()

In [ ]:
# helper for row_to_block()
def extract_problem_and_prefixcode(query: str) -> tuple[str, str]:
    """
    take in field query, return problem text and prefix_code
    e.g.

    * `problem_text`  - up before the first ```python fence, w/o closing ```
    * `prefix_code`   - rest
    """
    #print(query)
    m_open  = re.search(r"```python\s*", query)
    if not m_open:
        raise ValueError("opening ```python fence not found")

    m_block = re.search(r"```python\s*([\s\S]*?)```", query)
    if not m_block:
        raise ValueError("closing ``` fence not found")

    problem = query[: m_open.start()].rstrip()
    code    = m_block.group(1).rstrip()

    # keep the opening fence so prefix *starts* with ```python
    prefix  = "```python\n" + code
    return problem, prefix


# build one prompt block
def row_to_block(q, doc, k_io: int = 2, filled: bool = True) -> str:
    """
    Convert a q from ds to a prompt block, following ONE_PROMPT_BLOCK_TEMPLATE
    k_io: # i/o examples, default 2

    * If `filled` is False i.e. the target question, `{middle}` is left empty for the model to fill;
    o/w, {middle} is populated with completion code i.e.['completion] injected with docstrings
    """
    problem, prefix = extract_problem_and_prefixcode(q["query"])

    examples = "\n".join(
        f"Input:  {p['input']}\nOutput: {p['output']}"
        for p in q["input_output"][:k_io]
    )

    injected_code = inject_docstrings(q["completion"].strip(), doc)
    middle = injected_code if filled else ""

    # assemble block
    return ONE_PROMPT_BLOCK_TEMPLATE.format(
        problem=problem,
        examples=examples,
        prefix=prefix,
        middle=middle,
    )


In [ ]:
few_shot_part = "\n\n".join(
    row_to_block(r, doc=DOCS[i],filled=True) for i,r in enumerate(example_ds)
)

In [ ]:
print(few_shot_part)

<start>
You are given a 1-indexed integer array nums of length n.
An element nums[i] of nums is called special if i divides n, i.e. n % i == 0.
Return the sum of the squares of all special elements of nums.
 
Please complete the following python code precisely:
###
Input:  nums = [1,2,3,4]
Output: 21
Input:  nums = [2,7,1,19,18,3]
Output: 63
###
<fim_prefix>
```python
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
<fim_suffix>
<fim_middle>
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
        """
        Compute the sum of squares of the *special* elements in a 1-indexed array.
        
                A “special” element is one whose 1-based index divides the length
                of the array.  Formally, for an array of length *n*, an element
                `nums[i]` (using 1-based indexing) is special if `n % i == 0`.
        
                Parameters
                ----------
                nums : List[int]
                    A non-

### build complete prompt

In [ ]:
# ────────────────────────────────────────────────────────────────
# 1.  build the full prompt for a single question
# ────────────────────────────────────────────────────────────────
def make_prompt(
    few_shot_part: str,
    q        # one row from `ds`
             # formatter: row → str
) -> str:
    """
    Concatenate the system directive + few-shot examples + target question.
    """
    target_part = row_to_block(q, doc="",filled=False)

    return (
       "You are a Python programming assistant. "
        "Solve each new problem exactly as in previous examples. "
        "Do not output anything except the docstring and code that goes inside the provided "
        "class skeleton.\n"
        + few_shot_part
        + "\n\n"
        + target_part
        + "\n"
    )

# ────────────────────────────────────────────────────────────────
# 2.  send the prompt to the model and get back raw generation
# ────────────────────────────────────────────────────────────────
def generate_raw(
    prompt: str,
    tokenizer,
    model,
    device,
    max_new_tokens: int = 2048,
) -> str:
    """
    Encode → generate → decode → strip after <end>.
    Returns the model's raw textual response (still with both Solution classes).
    """
    def strip_after_end(text: str) -> str:
        return text.split("<end>", 1)[0].rstrip()

    # encode
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # generate
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.2, # param from paper eval expermients
            top_p=0.95,
            do_sample=True,
            eos_token_id=tokenizer.convert_tokens_to_ids("<end>"),
        )

    # decode only the new tokens
    gen_text = tokenizer.decode(output_ids[0][inputs.input_ids.shape[-1]:])
    return strip_after_end(gen_text)

# ────────────────────────────────────────────────────────────────
# 3.  pull the final Solution class out of that text
# ────────────────────────────────────────────────────────────────
def extract_solution(text: str) -> str:
    """
    Return the slice from the SECOND 'class Solution' onward.
    Fallbacks:
      • if only one hit → return from the first hit
      • if none → return the full text
    """
    hits = list(re.finditer(r'\bclass\s+Solution\b', text))
    if len(hits) >= 2:
        return text[hits[1].start():].lstrip()
    elif hits:
        return text[hits[0].start():].lstrip()
    else:
        return text.strip()



In [ ]:
injected = inject_docstrings(example_ds[2]['completion'], DOCS[2])
print(injected)

class BinaryIndexedTree:
    __slots__ = ["n", "c"]

    def __init__(self, n: int):
        self.n = n
        self.c = [-1] * (n + 1)

    def update(self, x: int, v: int):
        while x <= self.n:
            self.c[x] = max(self.c[x], v)
            x += x & -x

    def query(self, x: int) -> int:
        mx = -1
        while x:
            mx = max(mx, self.c[x])
            x -= x & -x
        return mx


class Solution:
    def maximumSumQueries(
        """
        Answer each query ``[xi, yi]`` with the maximum value of
                ``nums1[j] + nums2[j]`` over all indices *j* such that
                ``nums1[j] ≥ xi`` **and** ``nums2[j] ≥ yi``; return -1 if
                no index satisfies the constraints.
        
                The algorithm processes queries **offline**:
        
                1.  Sort item pairs *(nums1, nums2)* by ``nums1`` descending.
                2.  Sort queries by ``xi`` descending; iterate through them.
                3.  As we lower

In [ ]:

from typing import Tuple

def split_docsAndCode(src: str) -> Tuple[str, str]:
    """
    extract docstring from code, then return docstring + code w/ docstring removed
    Assume one and only one pair of docstring (valid for solution generated)
    """
    pat = re.compile(r'(?s)([rRuU]?[\'"]{3})(.*?)\1')   # DOTALL so .* spans lines
    matches = list(pat.finditer(src))

    if len(matches) != 1:
        raise ValueError("Expected exactly one docstring, found {}".format(len(matches)))

    m = matches[0]
    docstring = m.group(2)         # inner text (no surrounding quotes)

    # Cut out the docstring block from the source
    start, end = m.span()
    code_without = src[:start] + src[end:]              # preserve everything else

    # Strip leading/trailing whitespace lines so structure stays neat
    code_without = re.sub(r'\n{3,}', '\n\n', code_without.strip("\n")) + "\n"
    docstring = docstring.strip("\n")

    return code_without, docstring

## docstring + code gen

In [ ]:
def gen_pipeline(q) -> str:
    prompt = make_prompt(few_shot_part, q)
    raw = generate_raw(prompt, tokenizer, model, device)
    solution = extract_solution(raw)
    code, docs = split_docsAndCode(solution)
    return prompt, raw, solution, docs, code

### example usage of gen pipeline

In [ ]:
for e in example_ds:
    print(e["task_id"])

sum-of-squares-of-special-elements
find-the-longest-equal-subarray
maximum-sum-queries


In [ ]:
print(easy_ds[1]['task_id'])
print(medium_ds[1]['task_id'])
print(hard_ds[1]['task_id'])

find-maximum-number-of-string-pairs
count-zero-request-servers
sum-of-imbalance-numbers-of-all-subarrays


In [ ]:
# note first q of each ds already serves as example in prompt
prompt, raw, solution, docs, code = gen_pipeline(easy_ds[1]) # since select returns a ds; to get the entry, index once more
# print each
print(f"[PROMPT]:{prompt}")
print(f"[RAW]:{raw}")
print(f"[SOLUTION (DOCS + CODE)]:{solution}")
print(f"[SOLUTION-DOCS]:{docs}")
print(f"[SOLUTION-CODE]:{code}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[PROMPT]:You are a Python programming assistant. Solve each new problem exactly as in previous examples. Do not output anything except the docstring and code that goes inside the provided class skeleton.
<start>
You are given a 1-indexed integer array nums of length n.
An element nums[i] of nums is called special if i divides n, i.e. n % i == 0.
Return the sum of the squares of all special elements of nums.
 
Please complete the following python code precisely:
###
Input:  nums = [1,2,3,4]
Output: 21
Input:  nums = [2,7,1,19,18,3]
Output: 63
###
<fim_prefix>
```python
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
<fim_suffix>
<fim_middle>
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
        """
        Compute the sum of squares of the *special* elements in a 1-indexed array.
        
                A “special” element is one whose 1-based index divides the length
                of the array.  Formally, for an array of length *n*, an eleme

In [ ]:
# note first q of each ds already serves as example in prompt
prompt, raw, solution, docs, code = gen_pipeline(medium_ds[1]) # since select returns a ds; to get the entry, index once more
# print each
print(f"[PROMPT]:{prompt}")
print(f"[RAW]:{raw}")
print(f"[SOLUTION (DOCS + CODE)]:{solution}")
print(f"[SOLUTION-DOCS]:{docs}")
print(f"[SOLUTION-CODE]:{code}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[PROMPT]:You are a Python programming assistant. Solve each new problem exactly as in previous examples. Do not output anything except the docstring and code that goes inside the provided class skeleton.
<start>
You are given a 1-indexed integer array nums of length n.
An element nums[i] of nums is called special if i divides n, i.e. n % i == 0.
Return the sum of the squares of all special elements of nums.
 
Please complete the following python code precisely:
###
Input:  nums = [1,2,3,4]
Output: 21
Input:  nums = [2,7,1,19,18,3]
Output: 63
###
<fim_prefix>
```python
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
<fim_suffix>
<fim_middle>
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
        """
        Compute the sum of squares of the *special* elements in a 1-indexed array.
        
                A “special” element is one whose 1-based index divides the length
                of the array.  Formally, for an array of length *n*, an eleme

In [ ]:
# note first q of each ds already serves as example in prompt
prompt, raw, solution, docs, code = gen_pipeline(hard_ds[1]) # since select returns a ds; to get the entry, index once more
# print each
print(f"[PROMPT]:{prompt}")
print(f"[RAW]:{raw}")
print(f"[SOLUTION (DOCS + CODE)]:{solution}")
print(f"[SOLUTION-DOCS]:{docs}")
print(f"[SOLUTION-CODE]:{code}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[PROMPT]:You are a Python programming assistant. Solve each new problem exactly as in previous examples. Do not output anything except the docstring and code that goes inside the provided class skeleton.
<start>
You are given a 1-indexed integer array nums of length n.
An element nums[i] of nums is called special if i divides n, i.e. n % i == 0.
Return the sum of the squares of all special elements of nums.
 
Please complete the following python code precisely:
###
Input:  nums = [1,2,3,4]
Output: 21
Input:  nums = [2,7,1,19,18,3]
Output: 63
###
<fim_prefix>
```python
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
<fim_suffix>
<fim_middle>
class Solution:
    def sumOfSquares(self, nums: List[int]) -> int:
        """
        Compute the sum of squares of the *special* elements in a 1-indexed array.
        
                A “special” element is one whose 1-based index divides the length
                of the array.  Formally, for an array of length *n*, an eleme